# Advanced plotting

In this notebook, we explore some of the more advanced plotting features including:
 - plotting functions
 - controlling the plotting output (placement of labels, axes range, colors, figure size)
 - accessing the underlying matplotlib and bokeh interfaces to fine tune your plots

## Plotting functions

In addition to plotting data sets, QExPy allows you to add functions on your plots. Because QExPy knows how to handle numbers with uncertainties, the parameters of the functions can be Measurement Objects, and the resulting function can be plotted with error bands.

We start by importing the module and choosing the plot engine. 

In [1]:
import qexpy as q
q.plot_engine="bokeh" # choose bokeh or mpl

Loading BokehJS ...

To simply plot a function, we can define a Plot Object, and add a function to it. The function must be specified in the same format as a custom function that is being fit (see notebook 5).

We can also choose the number of points to use when plotting the function. By default, this is 100. If a function oscillates rapidly, then this may need to be increased. Of course, increasing this slows down the plotting. 

In [2]:
#We need to import numpy to define a function properly:
import numpy as np

#make an empty plot
fig1 = q.MakePlot()

#This will plot the function very coarsely
q.settings["plot_fcn_npoints"]=10

#define a python function to plot:
#let's plot a linear + gaussian:
def lineargauss(x, *pars):
    offset = pars[0]
    slope = pars[1]
    norm = pars[2]
    mean = pars[3]
    sigma = pars[4]
    
    linear = offset + slope*x
    gaussian = norm/np.sqrt(2*3.14)/sigma*np.exp(-(x-mean)**2/2/sigma**2)
    
    return linear + gaussian



#We can then add the function, naming it, giving the parameters, and specifying
#the range over which to plot it
fig1.add_function(lineargauss, name="linear + gauss", pars = [2,0.5,10,5,1],
                  color = 'saddlebrown',
                  x_range =[0,20])

#And we can then show it:
fig1.show()


### Plotting a function of parameters with uncertainties (and adding labels and changing the figure size)
We can replot the same function, but with parameters that have uncertainties. Suppose that we want to see the effect of an uncertainty in the normalization of the gaussian (the third parameters, pars[2]):

In [3]:
#Make a new figure:
fig2 = q.MakePlot()

#Set the range:
fig2.x_range=[0,10]

#Plot with more points:
q.settings["plot_fcn_npoints"]=100


#Store the paramaters for the function as a MeasurementArray
#Note that we chose only the third parameter to have a non-zero
#uncertainty
pars = q.MeasurementArray( [(2,0),
                            (0.5,0),
                            (10,2),
                            (5,0),
                            (1,0)])

#Add the function
fig2.add_function(lineargauss, pars = pars, name="uncertainty in norm",
                  color='darkkhaki',
                  x_range =[0,20])

#Let's also change the title of the axes:
fig2.labels["xtitle"]="energy [MeV]"
fig2.labels["ytitle"]="number of counts"
fig2.labels["title"]="location of the particle resonnance"

#Let's change the figure size, by giving dimensions in pixels:
fig2.dimensions_px = [800,400]

#show the figure
fig2.show()

## Use function plotting to show 2 different fits to the same dataset

By default, QExPy only shows the last fit to a dataset when plotting that dataset. However, we saw earlier that a dataset recalls all functions that were fit to it (and the results). Below, we generate a dataset programatically, then fit it to two functions and compare the results.


In [4]:
##############################
#Programatically generate data
##############################

#Generate some values around the x axis
xvals = np.linspace(0,20, 50)
#Generate values of y for those x values, normally distributed
#about our model:
#"true" parameters of the model:
true_pars = [1,0.1,20,8,1]
#the y values with some randomness:
yvals = np.random.normal(lineargauss(xvals, *true_pars),0.5)

##################
#Build the dataset
##################
xydata = q.XYDataSet(xdata = xvals, ydata = yvals, yerr = 0.5)
#Set the name of the dataset:
xydata.name = "Run10_Data"
xydata.xname = "Energy"
xydata.xunits = "MeV"
xydata.yname = "Counts"

#######################################
#Fit the data set to 2 different models
#######################################

#Let's fit it to a gaussian and to a linear+gaussian
#to see which one is best:

###Fit to a gaussian
#For the gaussian, we use the built in model:
#For a gaussian, it's best to give a guess for the parameters
gresults = xydata.fit("gaussian", parguess = [8,1,20])

###Fit to our custom model:
lgresults = xydata.fit(lineargauss, parguess = [1,1,1,8,1])

####################
#Display the results
####################

#Build a plot object:
fig3 = q.MakePlot(xydata)
#This will automatically display the last fit, so we add a function 
#for the first fit:
fig3.add_function(xydata.fit_function[0],
                  name = 'gaussian',
                  pars = xydata.fit_pars[0],
                  color = 'blue')
#The plot is a little busy if we show the fit results, so we don't show them:
fig3.show_fit_results = False
#The residuals will only be shown for the last fit
fig3.add_residuals()
fig3.show()


-----------------Fit results-------------------
Fit of  Run10_Data  to  gaussian
Fit parameters:
Run10_Data_gaussian_fit0_fitpars_mean = 8.1 +/- 0.2,
Run10_Data_gaussian_fit0_fitpars_sigma = 1.7 +/- 0.2,
Run10_Data_gaussian_fit0_fitpars_normalization = 36 +/- 4

Correlation matrix: 
[[  1.000e+00  -3.407e-08   9.613e-10]
 [ -3.407e-08   1.000e+00   5.774e-01]
 [  9.613e-10   5.774e-01   1.000e+00]]

chi2/ndof = 732.58/46
---------------End fit results----------------

-----------------Fit results-------------------
Fit of  Run10_Data  to  custom
Fit parameters:
Run10_Data_custom_fit1_fitpars_par0 = 0.9 +/- 0.2,
Run10_Data_custom_fit1_fitpars_par1 = 0.12 +/- 0.01,
Run10_Data_custom_fit1_fitpars_par2 = 21.1 +/- 1.0,
Run10_Data_custom_fit1_fitpars_par3 = 8.05 +/- 0.05,
Run10_Data_custom_fit1_fitpars_par4 = 1.07 +/- 0.05

Correlation matrix: 
[[ 1.    -0.847 -0.451  0.097 -0.292]
 [-0.847  1.     0.208 -0.114  0.135]
 [-0.451  0.208  1.    -0.024  0.648]
 [ 0.097 -0.114 -0.024  1.    -0.01

# Using the underlying bokeh interface
In the figure above, we would like to see the residuals from the first fit (the blue curve), since by default, QExPy will only show the results of the last fit. We can do this by modifying how we draw the figure. When we call show() on a Plot Object, QExPy will first create a figure, then populate it, then show it. We can modify the figure at any point during the show command, if we tell show() not to populate the figure (as this will erase the figure and start from scratch by default)

In [5]:
#First we populate the figure (with Bokeh, this starts from an empty figure)
fig3.populate_bokeh_figure()
#Then, we add the residuals from the first fit (fit index = 0)
fig3.bk_plot_dataset(xydata,residual=True, fit_index=0, color='blue')
#And now we show the figure, but ask QExPy not to populate it, since we
#already did
fig3.show(populate_figure=False)